# 20001955 - Trương Văn Thông
# NLP - Week 4

## Các thư viện sử dụng trong bài

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import urllib.error

from collections import Counter, defaultdict
import math
from math import log, sqrt

import nltk
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
# nltk.download('punkt')

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer

### Bài 1: Sử dụng BeautifulSoup và urllib để lấy các tin tự động trên trang: https://news.google.com/news/rss
- Viết các hàm xử lý để in ra màn hình 3 thông tin của mỗi tin: 
    - Tựa đề
    - Đường link
    - Ngày phát hành <br>
`Lưu kết quả vào 1 file theo từng dòng với cấu trúc như trên.`

In [5]:
def fetch_rss_data(url):
    try:
        news_list = []
        with urllib.request.urlopen(url) as response:
            rss_data = response.read()

        soup = BeautifulSoup(rss_data, 'xml')
        items = soup.find_all('item')

        for item in items:
            title = item.title.text if item.title else "Không có tiêu đề"
            link = item.link.text if item.link else "Không có đường link"
            pub_date = item.pubDate.text if item.pubDate else "Không có ngày phát hành"
            
            news_list.append((title, link, pub_date))
            print(f"Tựa đề: {title}")
            print(f"Đường Link: {link}")
            print(f"Ngày phát hành: {pub_date}")
            print("\n")

        return news_list

    except urllib.error.URLError as e:
        print(f"Lỗi kết nối mạng: {e}")
        return None
    except Exception as ex:
        print(f"Lỗi không xác định: {ex}")
        return None

def save_to_file(news_list, filename="rss_feed.txt"):
    try:
        with open(filename, "w", encoding="utf-8") as file:
            for title, link, pub_date in news_list:
                file.write(f"Tựa đề: {title}\n")
                file.write(f"Đường Link: {link}\n")
                file.write(f"Ngày phát hành: {pub_date}\n\n")

        print(f"Dữ liệu đã được lưu vào tệp: {filename}")
    except Exception as ex:
        print(f"Lỗi không xác định: {ex}")

if __name__ == "__main__":
    url = "https://news.google.com/news/rss"
    news_list = fetch_rss_data(url)
    if news_list:
        save_to_file(news_list)

Tựa đề: White House says 'there was no indication' DeSantis would snub Biden visit after Idalia - Fox News
Đường Link: https://news.google.com/rss/articles/CBMiW2h0dHBzOi8vd3d3LmZveG5ld3MuY29tL3BvbGl0aWNzL3doaXRlLWhvdXNlLW5vLWluZGljYXRpb24tZGVzYW50aXMtc251Yi1iaWRlbi12aXNpdC1pZGFsaWHSAQA?oc=5
Ngày phát hành: Sat, 02 Sep 2023 20:23:22 GMT


Tựa đề: North Korea says it has simulated a nuclear missile attack to warn US of 'nuclear war danger' - CNN
Đường Link: https://news.google.com/rss/articles/CBMiZmh0dHBzOi8vd3d3LmNubi5jb20vMjAyMy8wOS8wMi9hc2lhL25vcnRoLWtvcmVhLW51Y2xlYXItbWlzc2lsZS1hdHRhY2stc3RpbXVsYXRpb24taW50bC1obmsvaW5kZXguaHRtbNIBAA?oc=5
Ngày phát hành: Sun, 03 Sep 2023 01:18:00 GMT


Tựa đề: RAW VIDEO | Burning Man Festival: Attendees told to shelter in place and conserve food, water & fuel - KHOU 11
Đường Link: https://news.google.com/rss/articles/CCAiC0pSUk8xeXd3U2ZFmAEB?oc=5
Ngày phát hành: Sun, 03 Sep 2023 00:14:08 GMT


Tựa đề: Former New Mexico Governor, U.N. Ambassador Bill

### Bài 2: Cho dữ liệu như sau:
`Data = [{'Thanh pho': 'Hai Phong', 'Nhiet do': 32}, {'Thanh pho': 'Da Nang',
'Nhiet do': 29}, {'Thanh pho': 'Can Tho', 'Nhiet do': 34}]`

1. Hãy chuyển dữ liệu trên thành số sử dụng thư viện DictVectorizer

In [6]:
# Dữ liệu ban đầu
Data = [
    {'Thanh pho': 'Hai Phong', 'Nhiet do': 32},
    {'Thanh pho': 'Da Nang', 'Nhiet do': 29}, 
    {'Thanh pho': 'Can Tho', 'Nhiet do': 34}
]

vec = DictVectorizer()
vec_data = vec.fit_transform(Data).toarray()

print("Dữ liệu sau khi đã được chuyển đổi sử dụng thư viện DictVectorizer:")
print(vec_data)

Dữ liệu sau khi đã được chuyển đổi sử dụng thư viện DictVectorizer:
[[32.  0.  0.  1.]
 [29.  0.  1.  0.]
 [34.  1.  0.  0.]]


2. In ra tên các feature

In [7]:
print("Tên các feature:")
print(vec.get_feature_names_out())

Tên các feature:
['Nhiet do' 'Thanh pho=Can Tho' 'Thanh pho=Da Nang' 'Thanh pho=Hai Phong']


### Bài 3: Cho dữ liệu sau:
data= [{'word-2': 'con',
        'pos-2': 'DT',
        'word-1': 'mèo',
        'pos-1': 'NN',
        'word+1': 'trên',
        'pos+1': 'PP', }]

1. Hãy chuyển dữ liệu trên thành số sử dụng thư viện

In [8]:
data= [{'word-2': 'con',
        'pos-2': 'DT',
        'word-1': 'mèo',
        'pos-1': 'NN',
        'word+1': 'trên',
        'pos+1': 'PP', }]

vec = DictVectorizer()
vec_data = vec.fit_transform(data).toarray()

print("Dữ liệu sau khi đã được chuyển đổi:")
print(vec_data)

Dữ liệu sau khi đã được chuyển đổi:
[[1. 1. 1. 1. 1. 1.]]


2. In ra tên các feature

In [9]:
print("Tên các feature:")
print(vec.get_feature_names_out())

Tên các feature:
['pos+1=PP' 'pos-1=NN' 'pos-2=DT' 'word+1=trên' 'word-1=mèo' 'word-2=con']


### Bài 4: Cho dữ liệu sau:
dulieu=[
‘Hôm_nay tôi đi_học',
' Hôm_nay tôi đi_học ở trường',
' Hôm_nay tôi nghỉ ở nhà',
'Hôm_nay tôi có đi_học không?']

1. Hãy chuyển dữ liệu trên thành số sử dụng thư viện CountVectorizer

In [10]:
dulieu=[
'Hôm_nay tôi đi_học',
' Hôm_nay tôi đi_học ở trường',
' Hôm_nay tôi nghỉ ở nhà',
'Hôm_nay tôi có đi_học không?']

vec = CountVectorizer()
vec_data = vec.fit_transform(dulieu).toarray()

print("Chuyển dữ liệu thành số sử dụng thư viện CountVectorizer:")
print(vec_data)

Chuyển dữ liệu thành số sử dụng thư viện CountVectorizer:
[[0 1 0 0 0 0 1 1]
 [0 1 0 0 0 1 1 1]
 [0 1 0 1 1 0 1 0]
 [1 1 1 0 0 0 1 1]]


2. In ra tên của các feature

In [11]:
print("Tên các feature:")
print(vec.get_feature_names_out())

Tên các feature:
['có' 'hôm_nay' 'không' 'nghỉ' 'nhà' 'trường' 'tôi' 'đi_học']


3. Vào một câu bất kỳ, in ra vector giá trị số của câu đó dựa trên dữ liệu ở trên

In [12]:
# Câu bất kỳ
cau = "Hôm_nay tôi có đi_học không?"

# Chuyển câu thành vector
vec_cau = vec.transform([cau]).toarray()

print("Vector của câu '{}': ".format(cau))
print(vec_cau)

Vector của câu 'Hôm_nay tôi có đi_học không?': 
[[1 1 1 0 0 0 1 1]]


### Bài 5: Cho dữ liệu sau:
`Tôi là sinh viên trường Đại học Công nghiệp thành phố Hồ Chí Minh` <br>
Chuyển câu trên thành unigram, bigram và trigram sử dụng thư viện có sẵn và in
kết quả ra màn hình.

In [13]:
# Câu ban đầu
sentence = "Tôi là sinh viên trường Đại học Công nghiệp thành phố Hồ Chí Minh"

# Tokenize câu thành các từ
words = word_tokenize(sentence)

# Tạo unigram
unigrams = list(ngrams(words, 1))

# Tạo bigram
bigrams = list(ngrams(words, 2))

# Tạo trigram
trigrams = list(ngrams(words, 3))


print('Unigrams:', unigrams)
print("---"*50)
print('Bigrams:', bigrams)
print("---"*50)
print('Trigrams:', trigrams)


Unigrams: [('Tôi',), ('là',), ('sinh',), ('viên',), ('trường',), ('Đại',), ('học',), ('Công',), ('nghiệp',), ('thành',), ('phố',), ('Hồ',), ('Chí',), ('Minh',)]
------------------------------------------------------------------------------------------------------------------------------------------------------
Bigrams: [('Tôi', 'là'), ('là', 'sinh'), ('sinh', 'viên'), ('viên', 'trường'), ('trường', 'Đại'), ('Đại', 'học'), ('học', 'Công'), ('Công', 'nghiệp'), ('nghiệp', 'thành'), ('thành', 'phố'), ('phố', 'Hồ'), ('Hồ', 'Chí'), ('Chí', 'Minh')]
------------------------------------------------------------------------------------------------------------------------------------------------------
Trigrams: [('Tôi', 'là', 'sinh'), ('là', 'sinh', 'viên'), ('sinh', 'viên', 'trường'), ('viên', 'trường', 'Đại'), ('trường', 'Đại', 'học'), ('Đại', 'học', 'Công'), ('học', 'Công', 'nghiệp'), ('Công', 'nghiệp', 'thành'), ('nghiệp', 'thành', 'phố'), ('thành', 'phố', 'Hồ'), ('phố', 'Hồ', 'Chí'), ('Hồ', 

### Bài 6: Cho dữ liệu huấn luyện như sau:
```HTML
<s> tôi là IUH </s>
<s> IUH là tôi </s>
<s> IUH tôi học ở </s>
<s> IUH tôi đã học ở </s>
<s> tôi đã học ở IUH sao </s>
```
Giả sử chúng ta sử dụng bigram language model dựa trên dữ liệu đã huấn luyện ở
trên.

In [14]:
# Dữ liệu huấn luyện
sentences = [
    "<s> tôi là IUH </s>",
    "<s> IUH là tôi </s>",
    "<s> IUH tôi học ở </s>",
    "<s> IUH tôi đã học ở </s>",
    "<s> tôi đã học ở IUH sao </s>"
]

# Tokenization và Đếm Bigram
bigrams = []
unigrams = []
for sentence in sentences:
    tokens = sentence.split()
    unigrams.extend(tokens)
    bigrams.extend([(tokens[i], tokens[i + 1]) for i in range(len(tokens) - 1)])

bigram_counts = Counter(bigrams)
unigram_counts = Counter(unigrams)

# Tính Xác Suất
bigram_probabilities = defaultdict(float)
for (w1, w2), count in bigram_counts.items():
    bigram_probabilities[(w1, w2)] = count / unigram_counts[w1]

# In kết quả
for (w1, w2), prob in bigram_probabilities.items():
    print(f"P({w2}|{w1}) = {prob}")

P(tôi|<s>) = 0.4
P(là|tôi) = 0.2
P(IUH|là) = 0.5
P(</s>|IUH) = 0.2
P(IUH|<s>) = 0.6
P(là|IUH) = 0.2
P(tôi|là) = 0.5
P(</s>|tôi) = 0.2
P(tôi|IUH) = 0.4
P(học|tôi) = 0.2
P(ở|học) = 1.0
P(</s>|ở) = 0.6666666666666666
P(đã|tôi) = 0.4
P(học|đã) = 1.0
P(IUH|ở) = 0.3333333333333333
P(sao|IUH) = 0.2
P(</s>|sao) = 1.0


1. Hãy cho biết từ có thể xuất hiện tiếp theo theo model huấn luyện ở trên trong các câu sau là gì?
```HTML
(1) <s> IUH . . .
(2) <s> IUH tôi học . . .
(3) <s> IUH tôi là IUH . . .
(4) <s> tôi đã học . . .
```

In [15]:
# Dự đoán từ tiếp theo
def predict_next_word(sentence):
    tokens = sentence.split()
    last_token = tokens[-1]
    possible_next_words = [w2 for (w1, w2) in bigram_probabilities.keys() if w1 == last_token]
    
    if not possible_next_words:
        return "Không có từ nào có thể xuất hiện tiếp theo."
    
    # Trả về từ có xác suất xuất hiện cao nhất (trong trường hợp này, tất cả đều có xác suất bằng nhau)
    return possible_next_words[0]  # Điều này giả định rằng tất cả các từ có xác suất xuất hiện tiếp theo bằng nhau

# Test
test_sentences = [
    "<s> IUH",
    "<s> IUH tôi học",
    "<s> IUH tôi là IUH",
    "<s> tôi đã học"
]

for i, sentence in enumerate(test_sentences, start=1):
    next_word = predict_next_word(sentence)
    print(f"({i}) '{sentence} ...', từ có thể xuất hiện tiếp theo là: '{next_word}'")

(1) '<s> IUH ...', từ có thể xuất hiện tiếp theo là: '</s>'
(2) '<s> IUH tôi học ...', từ có thể xuất hiện tiếp theo là: 'ở'
(3) '<s> IUH tôi là IUH ...', từ có thể xuất hiện tiếp theo là: '</s>'
(4) '<s> tôi đã học ...', từ có thể xuất hiện tiếp theo là: 'ở'


2. Trong các câu sau, câu nào cho xác suất cao nhất với mô hình đã huấn luyện ở
trên?
```HTML
(5) <s> IUH tôi đã tôi học </s>
(6) <s> IUH tôi là </s>
(7) <s> tôi đã học IUH tôi là </s>
```

In [16]:
def calculate_sentence_probability(sentence, bigram_probabilities):
    tokens = sentence.split()
    log_prob = 0.0  # Sử dụng log để tính
    
    for i in range(1, len(tokens)):
        w1, w2 = tokens[i - 1], tokens[i]
        prob = bigram_probabilities.get((w1, w2), 0)  # 0 nếu bigram không tồn tại trong tập dữ liệu
        if prob == 0:  # Nếu bigram không tồn tại, câu đó có xác suất là 0
            return 0
        
        log_prob += math.log(prob)
    
    return math.exp(log_prob)  # Convert lại từ log xác suất sang xác suất

# Các câu cần tính xác suất
test_sentences = [
    "<s> IUH tôi đã tôi học </s>",
    "<s> IUH tôi là </s>",
    "<s> tôi đã học IUH tôi là </s>"
]

# Tính xác suất và in ra
for sentence in test_sentences:
    prob = calculate_sentence_probability(sentence, bigram_probabilities)
    print(f"Xác suất của câu '{sentence}' là: {prob}")


Xác suất của câu '<s> IUH tôi đã tôi học </s>' là: 0
Xác suất của câu '<s> IUH tôi là </s>' là: 0
Xác suất của câu '<s> tôi đã học IUH tôi là </s>' là: 0


3. Hãy tính complexity của câu sau:
```HTML
<s> tôi đã học ở IUH
```

In [17]:
def calculate_sentence_complexity(sentence, bigram_probabilities):
    tokens = sentence.split()
    N = len(tokens)
    log_prob = 0.0
    
    for i in range(1, N):
        w1, w2 = tokens[i - 1], tokens[i]
        prob = bigram_probabilities.get((w1, w2), 0)
        
        if prob == 0:
            return float('inf')  # return infinity nếu câu có xác suất là 0
        
        log_prob += math.log(prob)
        
    complexity = math.exp(-log_prob / N)
    
    return complexity

# Câu cần tính complexity
sentence = "<s> tôi đã học ở IUH"

# Tính complexity và in ra
complexity = calculate_sentence_complexity(sentence, bigram_probabilities)
print(f"Complexity của câu '{sentence}' là: {complexity}")


Complexity của câu '<s> tôi đã học ở IUH' là: 1.6299222137747946


4. Tính xác suất bigram dựa trên mô hình huấn luyện ở trên cho các từ sau:
```HTML
P(học |<s>)
P(học | IUH)
P(IUH|<s>)
P(IUH| học)
P(tôi|IUH)
P(tôi|học)
P(học|tôi)
```

In [18]:
# Tính xác suất bigram
def calc_bigram_prob(w1, w2):
    return bigram_counts.get((w1, w2), 0) / unigram_counts.get(w1, 0)

# Tính và in ra các xác suất bigram cần tìm
bigrams_to_check = [
    ("<s>", "học"),
    ("IUH", "học"),
    ("<s>", "IUH"),  
    ("học", "IUH"),
    ("IUH", "tôi"), 
    ("học", "tôi"),  
    ("tôi", "học")   
]

for w1, w2 in bigrams_to_check:
    prob = calc_bigram_prob(w1, w2)
    print(f"P({w2} | {w1}) = {prob}")


P(học | <s>) = 0.0
P(học | IUH) = 0.0
P(IUH | <s>) = 0.6
P(IUH | học) = 0.0
P(tôi | IUH) = 0.4
P(tôi | học) = 0.0
P(học | tôi) = 0.2


5. Hãy tính xác suất của câu sau:
```HTML
(8) <s> tôi đã học ở IUH sao
(9) <s> IUH tôi đã học
```

In [19]:
# Tính xác suất của một câu
def calc_sentence_prob(sentence, unigram_counts, bigram_counts):
    words = sentence.split()
    prob = 1.0
    for w1, w2 in zip(words, words[1:]):
        bigram_prob = bigram_counts.get((w1, w2), 0) / unigram_counts.get(w1, 0)
        prob *= bigram_prob
        if bigram_prob == 0:
            return 0
    return prob

# Các câu cần tính xác suất
sentences_to_check = [
    "<s> tôi đã học ở IUH sao",
    "<s> IUH tôi đã học"
]

# Tính và in ra xác suất
for sentence in sentences_to_check:
    prob = calc_sentence_prob(sentence, unigram_counts, bigram_counts)
    print(f"Xác suất của câu '{sentence}' là: {prob}")

Xác suất của câu '<s> tôi đã học ở IUH sao' là: 0.01066666666666667
Xác suất của câu '<s> IUH tôi đã học' là: 0.096


### Bài 7: Cho dữ liệu như ở bài 4:
dulieu=[
‘Hôm_nay tôi đi_học',
' Hôm_nay tôi đi_học ở trường',
' Hôm_nay tôi nghỉ ở nhà',
'Hôm_nay tôi có đi_học không?'] <br><br>
Với các định nghĩa như sau về TF-IDF: <br><br>
Tf means **term-frequency** while tf–idf means term-frequency times **inverse document-frequency**:
$$tf-idf(t, d) = tf(t, d) * log(N/(df + 1))$$
✓ *tf(t,d) = count of t in d / number of words in d* <br>
✓ *df(t) = occurrence of t in documents*<br>
✓ *idf(t) = log(N/(df + 1))*<br>
• *t — term (word), d — document (set of words)*<br>
• *n — count of corpus, corpus — the total document set*<br><br>
Kết quả sẽ được chuẩn hóa bằng công thức Euclidean: <br>

$$v_{norm} = \frac{v}{\left \| v \right \|_2} = \frac{v}{\sqrt{v_1^2 + v_2^2 + ... + v_n^2}}$$

<br>Hãy tự tính `TF-IDF` của văn bản đã cho theo các công thức ở trên

In [20]:
# Dữ liệu
dulieu = [
    'Hôm_nay tôi đi_học',
    'Hôm_nay tôi đi_học ở trường',
    'Hôm_nay tôi nghỉ ở nhà',
    'Hôm_nay tôi có đi_học không'
]

Tính Term Frequency (TF):

In [21]:
# Tính TF
tf = {}
for idx, doc in enumerate(dulieu):
    word_count = Counter(doc.split())
    total_words = len(doc.split())
    tf[idx] = {word: count / total_words for word, count in word_count.items()}
    
print("TF:")
print(tf)

TF:
{0: {'Hôm_nay': 0.3333333333333333, 'tôi': 0.3333333333333333, 'đi_học': 0.3333333333333333}, 1: {'Hôm_nay': 0.2, 'tôi': 0.2, 'đi_học': 0.2, 'ở': 0.2, 'trường': 0.2}, 2: {'Hôm_nay': 0.2, 'tôi': 0.2, 'nghỉ': 0.2, 'ở': 0.2, 'nhà': 0.2}, 3: {'Hôm_nay': 0.2, 'tôi': 0.2, 'có': 0.2, 'đi_học': 0.2, 'không': 0.2}}


Tính Document Frequency (DF) và Inverse Document Frequency (IDF):


In [22]:
# Tính DF
df = Counter()
for doc in dulieu:
    for word in set(doc.split()):
        df[word] += 1

# Tính IDF và TF-IDF
N = len(dulieu)
idf = {word: log(N / (df[word] + 1)) for word in df.keys()}
# Tính TF-IDF
tf_idf = {}
for idx, doc in tf.items():
    tf_idf[idx] = {word: tf_word * idf[word] for word, tf_word in doc.items()}

print(tf_idf)

{0: {'Hôm_nay': -0.07438118377140324, 'tôi': -0.07438118377140324, 'đi_học': 0.0}, 1: {'Hôm_nay': -0.044628710262841945, 'tôi': -0.044628710262841945, 'đi_học': 0.0, 'ở': 0.05753641449035617, 'trường': 0.13862943611198905}, 2: {'Hôm_nay': -0.044628710262841945, 'tôi': -0.044628710262841945, 'nghỉ': 0.13862943611198905, 'ở': 0.05753641449035617, 'nhà': 0.13862943611198905}, 3: {'Hôm_nay': -0.044628710262841945, 'tôi': -0.044628710262841945, 'có': 0.13862943611198905, 'đi_học': 0.0, 'không': 0.13862943611198905}}


Chuẩn hóa TF-IDF (Euclidean Normalization)

In [23]:
# Chuẩn hóa TF-IDF
normalized_tf_idf = {}
for idx, doc in tf_idf.items():
    norm = sqrt(sum([value ** 2 for value in doc.values()]))
    normalized_tf_idf[idx] = {word: value / norm for word, value in doc.items()}

# In kết quả
for idx, doc in normalized_tf_idf.items():
    print(f"Document {idx+1}: {doc}")

Document 1: {'Hôm_nay': -0.7071067811865476, 'tôi': -0.7071067811865476, 'đi_học': 0.0}
Document 2: {'Hôm_nay': -0.27408992070678545, 'tôi': -0.27408992070678545, 'đi_học': 0.0, 'ở': 0.35336336615008035, 'trường': 0.851401058372011}
Document 3: {'Hôm_nay': -0.2086955305784808, 'tôi': -0.2086955305784808, 'nghỉ': 0.6482675289694743, 'ở': 0.269055334087166, 'nhà': 0.6482675289694743}
Document 4: {'Hôm_nay': -0.21668588308533965, 'tôi': -0.21668588308533965, 'có': 0.673087830874639, 'đi_học': 0.0, 'không': 0.673087830874639}
